# FEniCSx Tutorial

In this tutorial, we will be looking at the FEniCSx computing platform and seeing how it assists with FEA analysis. As we have seen in class, our starting point was going through simple elastic solvers and Newtonian methods. We will similarily begin this tutorial with an implementation of Linear Elasticity in 2 dimensions, or simulating a downward force on an elastic beam. Before getting into the problem, please ensure that FEniCSx and DolfinX are installed on your computer. For assistance with installation, please refer to the 'readme.md' for tips on installing the platform, creating a conda environment, activating it (also in your terminal/visual studio session), and running the Jupyter Notebook.

## Library Imports

To ensure that FEniCSx runs properly, please run each Jupyter notebook cell in order. If you run any cells ahead of others, the code might error out. First and foremost, we call our imports, the main ones being Numpy, DolfinX, UFL. 

Here:


In [ ]:
import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem, plot
import ufl
import pyvista
from dolfinx.plot import create_vtk_mesh